In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import copy

In [ ]:
dataset=pd.read_csv('virus_prediction.csv')

In [ ]:
d1=copy.copy(dataset)

In [ ]:
del dataset['SlNo'],dataset['FarmerEduaction'],dataset['FarmerAge'],dataset['GPSCordinate'],dataset['Zone']

In [ ]:
q=dataset

In [ ]:
del q['Temperature'],q['pH'],q['Salinity']

In [ ]:
len(q.columns)

In [ ]:
X_features = q.iloc[:,0:37]
y_label = q.iloc[:, 38]

In [ ]:
len(dataset.columns)

In [ ]:
q.corr()

In [ ]:
c = q.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort")

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
so[::-1]

In [ ]:
del q['CrabFench'],q['FootBath'],q['Aerator'],q['Reservoir'],q['LimitedAccess'],q['WaterSource_IndirectNatural']

In [ ]:
len(q.columns)

In [ ]:
r=q.corr()

In [ ]:
r['VirusDetected']

In [ ]:
non_target=q.iloc[:,0:31]
target=q.iloc[:,32]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=1, max_depth=10)
df=pd.get_dummies(non_target)
model.fit(df,target)

In [ ]:
features = df.columns
importances = model.feature_importances_
indices = np.argsort(importances)[-12:]  # top 12 features
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
k=[features[i] for i in indices]

In [ ]:
k

In [ ]:
a=[]
for i in k:
    a.append(q[i])

In [ ]:
subset=q[['SourceOfFingerling',
 'CanalDepth_ft',
 'WaterSource_DirectNatural',
 'PeriodOfFallow',
 'FeedType',
 'StockingAge_Days',
 'WaterExchangeFrequency',
 'InvolveWithShrimpFarming',
 'StockingDensity_PL/40MeterSquare',
 'GherDepth_ft',
 'Area_ha',
 'PreviousPrevalence(%)',
         'VirusDetected']]

In [ ]:
subset.corr()

In [ ]:
import random
split_ratio=0.70

def split_train_test(data, split_ratio, train, test):
    for index,row in data.iterrows():
        if(random.random()>split_ratio):
            test.append(row)
        else:
            train.append(row)
        
train=[]
test=[]

split_train_test(subset, split_ratio, train, test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
xtrain=[]
ytrain=[]
for x in train:
    xtrain.append(x[:12])
    ytrain.append(x[12])

In [ ]:
xtest=[]
ytest=[]
for x in test:
    xtest.append(x[:12])
    ytest.append(x[12])

In [ ]:
from sklearn import metrics
DT = DecisionTreeClassifier(max_depth=4)
DT = DT.fit(xtrain,ytrain)
pred_test = DT.predict(xtest)
pred_train = DT.predict(xtrain)
print("Accuracy for Decision tree, Random Forest, KNN and Logistic Regression models\n")
print("Decision tree")
print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
print("")

from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(max_depth=4)
RFC = RFC.fit(xtrain,ytrain)
pred_test = RFC.predict(xtest)
pred_train = RFC.predict(xtrain)
print("Random forest classifier")
print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
print("")

from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=3)
KNN = KNN.fit(xtrain,ytrain)
pred_test = KNN.predict(xtest)
pred_train = KNN.predict(xtrain)
print("K nearest neighbour")
print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
print("")

from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR = LR.fit(xtrain,ytrain)
pred_test = LR.predict(xtest)
pred_train = LR.predict(xtrain)
print("Logistic Regression")
print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
print("")



In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4)

In [ ]:
a=subset.iloc[:,0:12]
b=subset.iloc[:,12]

DTSKF = DecisionTreeClassifier(max_depth=4)
RFCSKF = RandomForestClassifier(max_depth=4)
KNNSKF = KNeighborsClassifier(n_neighbors=3)
LRSKF = LogisticRegression(solver='liblinear')

dt_train_accuracy=[]
dt_test_accuracy=[]

rf_train_accuracy=[]
rf_test_accuracy=[]

knn_train_accuracy=[]
knn_test_accuracy=[]

lr_train_accuracy=[]
lr_test_accuracy=[]

for train_index, test_index in skf.split(a,b):
    #print(train_index,test_index)
    x_train=[]
    x_test=[]
    y_train=[]
    y_test=[]
    for i in train_index:
        x_train.append(a.iloc[i])
        y_train.append(b.iloc[i])
        
    for i in test_index:
        x_test.append(a.iloc[i])
        y_test.append(b.iloc[i])
        

    DTSKF = DTSKF.fit(x_train,y_train)
    pred_train = DTSKF.predict(x_train)
    pred_test = DTSKF.predict(x_test)
    #print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
    #print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
    dt_train_accuracy.append(metrics.accuracy_score(y_train, pred_train))
    dt_test_accuracy.append(metrics.accuracy_score(y_test, pred_test))
    
    RFCSKF = RFCSKF.fit(x_train,y_train)
    pred_train = RFCSKF.predict(x_train)
    pred_test = RFCSKF.predict(x_test)
    #print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
    #print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
    rf_train_accuracy.append(metrics.accuracy_score(y_train, pred_train))
    rf_test_accuracy.append(metrics.accuracy_score(y_test, pred_test))
    
    KNNSKF = KNNSKF.fit(x_train,y_train)
    pred_train = KNNSKF.predict(x_train)
    pred_test = KNNSKF.predict(x_test)
    #print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
    #print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
    knn_train_accuracy.append(metrics.accuracy_score(y_train, pred_train))
    knn_test_accuracy.append(metrics.accuracy_score(y_test, pred_test))
    
    LRSKF = LRSKF.fit(x_train,y_train)
    pred_train = LRSKF.predict(x_train)
    pred_test = LRSKF.predict(x_test)
    #print("Train Accuracy:",metrics.accuracy_score(ytrain, pred_train))
    #print("Test Accuracy:",metrics.accuracy_score(ytest, pred_test))
    lr_train_accuracy.append(metrics.accuracy_score(y_train, pred_train))
    lr_test_accuracy.append(metrics.accuracy_score(y_test, pred_test))
    

print("Accuracy for Decision tree, Random Forest Classifier, KNN and Logistic Regression models using stratified K fold cross validation\n")
print("Decision tree")
print("training accuracy : "+str(sum(dt_train_accuracy)/4))
print("testing accuracy  : "+str(sum(dt_test_accuracy)/4))
print("")
print("Random forest classifier")
print("training accuracy : "+str(sum(rf_train_accuracy)/4))
print("testing accuracy  : "+str(sum(rf_test_accuracy)/4))
print("")
print("K nearest neighbour")
print("training accuracy : "+str(sum(knn_train_accuracy)/4))
print("testing accuracy  : "+str(sum(knn_test_accuracy)/4))
print("")
print("Logistic Regression")
print("training accuracy : "+str(sum(lr_train_accuracy)/4))
print("testing accuracy  : "+str(sum(lr_test_accuracy)/4))

In [ ]:
# Graph section

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
# Without stratified k fold

# Decision tree
dt_train_accuracy=[81.0, 77.7, 84.5, 77.3, 84.1]
dt_test_accuracy=[57.6, 60.5, 62.0, 70.0, 57.3]

# Random forest classifier
rf_train_accuracy=[85.1, 87.8, 86.8, 85.8, 86.7]
rf_test_accuracy=[71.7, 75.0, 67.2, 68.5, 70.6]

# KNN
knn_train_accuracy=[74.3, 77.0, 76.5, 77.3, 75.9]
knn_test_accuracy=[63.5, 69.7, 50.0, 67.1, 52.0]

# Logistic Regression
lr_train_accuracy=[70.9, 69.4, 73.7, 73.0, 66.4]
lr_test_accuracy=[56.4, 67.1, 55.4, 61.4, 66.6]

instances=[1,2,3,4,5]

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Decision Tree")
plt.plot(instances,dt_train_accuracy)
plt.plot(instances,dt_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()

plt.figure(figsize=(8,6))
plt.title("Random Forest Classifier")
plt.plot(instances,rf_train_accuracy)
plt.plot(instances,rf_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()

plt.figure(figsize=(8,6))
plt.title("KNN")
plt.plot(instances,knn_train_accuracy)
plt.plot(instances,knn_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()

plt.figure(figsize=(8,6))
plt.title("Logistic Regression")
plt.plot(instances,lr_train_accuracy)
plt.plot(instances,lr_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()


In [ ]:
# With Stratified K fold

# Decision tree
dt_skf_train_accuracy=[81.2, 81.2, 81.2, 81.2, 81.2]
dt_skf_test_accuracy=[60.0, 61.3, 60.4, 60.4, 60.4]

# Random forest classifier
rf_skf_train_accuracy=[84.8, 84.4, 83.9, 85.6, 84.8]
rf_skf_test_accuracy=[68.6, 69.5, 69.5, 68.7, 69.0]

# KNN
knn_skf_train_accuracy=[76.2, 76.2, 76.2, 76.2, 76.2]
knn_skf_test_accuracy=[56.6, 56.6, 56.7, 56.6, 56.6]

# Logistic Regression
lr_skf_train_accuracy=[70.5, 70.5, 70.5, 70.5, 70.5]
lr_skf_test_accuracy=[61.8, 61.8, 61.8, 61.8, 61.8]

instances=[1,2,3,4,5]

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Decision Tree")
plt.plot(instances,dt_skf_train_accuracy)
plt.plot(instances,dt_skf_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()

plt.figure(figsize=(8,6))
plt.title("Random Forest Classifier")
plt.plot(instances,rf_skf_train_accuracy)
plt.plot(instances,rf_skf_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()

plt.figure(figsize=(8,6))
plt.title("KNN")
plt.plot(instances,knn_skf_train_accuracy)
plt.plot(instances,knn_skf_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()

plt.figure(figsize=(8,6))
plt.title("Logistic Regression")
plt.plot(instances,lr_skf_train_accuracy)
plt.plot(instances,lr_skf_test_accuracy)
plt.ylim(0,100)
plt.xlabel("instances")
plt.ylabel("accuracy")
plt.legend(['train accuracy', 'test accuracy'])
plt.show()

In [ ]:
# Conclusion
# 1. Random forest classifier is the best model in terms of both test and train accuracy.
# 2. K nearest neighbour performs worst among all the 4 models for disease prediction.
# 3. Adding stratified K fold validation reduces the variance of train/test accuracies for all 4 models.
# 4. The gap between train accuracy and testing accuracy is least for logistic regression indicating that LR model 
#    works well with unseen data.


In [ ]:
def convert(n):
    if(n==0.0 or n==0):
        return "No Disease"
    else:
        return "Disease"


instance_list=[]
import random
for x,y in subset.iteritems():
    if(x!='VirusDetected'):
        if(y.dtypes=="int"):
            instance_list.append(random.randint(min(y),max(y)))
        else:
            instance_list.append(random.uniform(min(y),max(y)))
            
single_instance=np.array(instance_list).reshape(1,-1)

print("Prediction for simple models -")
print("Decision tree: "+convert(DT.predict(single_instance)))
print("Random forest classifier: "+convert(RFC.predict(single_instance)))
print("KNN: "+convert(KNN.predict(single_instance)))
print("Logistic Regression: "+convert(LR.predict(single_instance)))

print("\nPrediction for models after adding stratified k fold cross validation -")
print("Decision tree: "+convert(DTSKF.predict(single_instance)))
print("Random forest classifier: "+convert(RFCSKF.predict(single_instance)))
print("KNN: "+convert(KNNSKF.predict(single_instance)))
print("Logistic Regression: "+convert(LRSKF.predict(single_instance)))